In [1]:
import numpy as np
import json
import pandas as pd
# import tensorflow
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time
from collections import Counter
from pipe import transform_text_func,FeatureExtractor, ImputeNA, CategoricalEncoding,text
from scipy.sparse import hstack
from sklearn.linear_model import RidgeClassifier,Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score, make_scorer,mean_squared_error
from sklearn.preprocessing import StandardScaler
from nltk.stem import PorterStemmer
from scipy.sparse import csr_matrix
import logging
from sklearn.pipeline import make_pipeline, make_union 
from keras.layers import Input, Dropout, Dense, concatenate, GRU, Embedding, Flatten, Activation, BatchNormalization, PReLU
from keras.initializers import he_uniform
from keras.layers import Conv1D
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.optimizers import Adam, SGD
from keras.models import Model
import gc
# from gensim import corpora,models,similarities
# import gensim

from sklearn.externals import joblib

/home/kevindong1994/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
import os
os.listdir()

# word ridge

In [2]:
train = pd.read_csv('training_data.csv',header= 0 ,delimiter='\t|\n')
print('-------generate interaction feature between cate-------')
s = time.time()
a = ['age_cat','sex','stay_cat','lang','er','category']

for i,ai in enumerate(a):
    for j,bj in enumerate(a):
        if i<j:
            x = train[ai]
            y = train[bj]
            t = []
            for l in range(train.shape[0]):
                t.append(str(x[l])+' '+ str(y[l]))
            train[ai+'_'+bj] = t
print('time elapsed: ', time.time()-s)

x_total = list(train.comment)
y_total = list(train.score)

print('-------calculate uppercase prob to features-------')
s = time.time()
uppercase = []
for i in x_total:
    length = len(i.split())
    tmp = []
    for j in i:
        if j.isupper():
            tmp.append(j)
    uppercase.append(len(tmp)/length)
uppercase = np.array(uppercase).reshape(-1,1)
scaler = StandardScaler()
new_up = scaler.fit_transform(uppercase)
print('time elapsed: ', time.time()-s)

print('-------choose specific stop words-------')
s=time.time()
x_lower = [sublist.lower() for sublist in x_total]
# punctuation = string.punctuation.replace
# regex = re.compile('[' +re.escape(punctuation) +']')
def tokenize(text):
#     text = regex.sub(" ", text) # remove punctuation
    text = text.replace('.',' ')
#     text = text.replace(',',' ')
#     text = text.replace('&',' and ')
#     text = text.replace('/',' or ')
    text = text.split()
    return(text)
x_lower = [tokenize(i) for i in x_lower]
x_unlist = []
for i in x_lower:
    x_unlist += i
vocab_dic = Counter(x_unlist)
stopwords_num = 250
# print([i[0] for i in vocab_dic.most_common(stopwords_num)],'\n\n')
print('total vocab: ',len(vocab_dic.most_common()))
maxfeature = len([i[0] for i in vocab_dic.most_common() if i[1]>0])
print('vocab size frequency > 1: ', maxfeature)

stop = [i[0] for i in vocab_dic.most_common(stopwords_num)]
from itertools import compress
x_5level = [list(compress(x_lower, list(np.array(y_total)==i))) for i in np.unique(y_total)]

x_5level_unlist = [[],[],[],[],[],[],[],[],[],[],[]]
for i in range(len(x_5level)):
    for j in x_5level[i]:
        x_5level_unlist[i] += j
multilevel_vocab = []
for i in range(11):
    multilevel_vocab.append(Counter(x_5level_unlist[i]))


from collections import defaultdict
multilevel_stop = defaultdict(list)
for i in range(11):
    tt = len(x_5level_unlist[i])
    for j in stop:
        multilevel_stop[j].append(multilevel_vocab[i][j]/tt)

stop_var = [(key,np.std(value)*1000) for key,value in multilevel_stop.items() ]
stop = [i[0] for i in stop_var if i[1]<0.1]

print('time elapsed: ', time.time()-s)



def tokenize(text):
    try:
        punctuation = string.punctuation.replace('#','')
#         .replace('!','').replace('?','').replace("'",'').replace("/",'').replace("@",'').replace('"','')
        regex = re.compile('[' +re.escape(punctuation) +']')
        text = regex.sub(" ", text) # remove punctuation
        text = text.replace('#1','')
        text = text.replace('#2','')
        text = text.replace('#3','')
        text = text.replace('#4','')
        text = text.replace('#5','')
        text = text.replace('#6','')
        text = text.replace('#7','')
        text = text.replace('#8','')
        text = text.replace('#9','')
#         text = text.replace('/w',' with ')
#         text = text.replace('/',' ')
#         text = text.replace('&',' and ')
#         text = text.replace("/",' or ')
#         text = text.replace('""',' sarcasm ')
#         text = text.replace("'d",' ')
#         text = text.replace("'s",' ')
#         text = text.replace("'re",' ')
#         text = text.replace("'ll",' ')
#         text = text.replace("'ve",' ')    

#         text = text.replace('.',' ')

        ps = PorterStemmer()
        tokens = []
        tokens_ = [s.split() for s in sent_tokenize(text)]
        for token_by_sent in tokens_:
            tokens += token_by_sent   
        filtered_tokens = [ps.stem(w.lower()) for w in tokens]
        return filtered_tokens
    except TypeError as e: print(text,e)

print('-------transform to features-------')
s=time.time()
onehot_list = ['age_cat', 'sex', 'stay_cat', 'lang', 'er','age_cat_sex', 'age_cat_stay_cat',
       'age_cat_lang', 'age_cat_er', 'age_cat_category', 'sex_stay_cat',
       'sex_lang', 'sex_er', 'sex_category', 'stay_cat_lang', 'stay_cat_er',
       'stay_cat_category', 'lang_er', 'lang_category', 'er_category']
onehot_pipeline = make_pipeline(FeatureExtractor(onehot_list),
                                CategoricalEncoding('OneHot'),
                                )
descrip_pipeline = make_pipeline(FeatureExtractor('comment'),
                                text(method='tfidf', ngram = 3, max_f = maxfeature, 
                                     binary = False, stopwords=stop,tokenizer=tokenize,analyzer ='word'))

feature_union = make_union(
    onehot_pipeline,
    descrip_pipeline
)
X = feature_union.fit_transform(train)
print('time elapsed: ', time.time()-s)

print('-------add length and upper prob to features-------')
s = time.time()
length = np.array([(X[i,]!=0).sum() for i in range(X.shape[0])]).reshape(-1,1)
scaler = StandardScaler()
new_l = scaler.fit_transform(length)
X = hstack([X,new_l],format='csr')
X = hstack([X,new_up],format='csr')

print('X shape: ',X.shape)
print('time elapsed: ', time.time()-s)

x_train,y_train = X, y_total

print('--------------Word ridge data complete--------------')

/home/kevindong1994/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


-------generate interaction feature between cate-------
time elapsed:  41.4511284828186
-------calculate uppercase prob to features-------
time elapsed:  1.3654165267944336
-------choose specific stop words-------
total vocab:  45314
vocab size frequency > 1:  45314
time elapsed:  2.1865382194519043
-------transform to features-------
time elapsed:  68.8904595375061
-------add length and upper prob to features-------


/home/kevindong1994/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


X shape:  (151824, 45760)
time elapsed:  35.504518032073975
--------------Word ridge data complete--------------


In [6]:
train = pd.read_csv('test_data.csv',header= 0 ,delimiter='\t|\n')
print('-------generate interaction feature between cate-------')
s = time.time()
a = ['age_cat','sex','stay_cat','lang','er','category']

for i,ai in enumerate(a):
    for j,bj in enumerate(a):
        if i<j:
            x = train[ai]
            y = train[bj]
            t = []
            for l in range(train.shape[0]):
                t.append(str(x[l])+' '+ str(y[l]))
            train[ai+'_'+bj] = t
print('time elapsed: ', time.time()-s)

x_total = list(train.comment)
#y_total = list(train.score)

print('-------calculate uppercase prob to features-------')
s = time.time()
uppercase = []
for i in x_total:
    length = len(i.split())
    tmp = []
    for j in i:
        if j.isupper():
            tmp.append(j)
    uppercase.append(len(tmp)/length)
uppercase = np.array(uppercase).reshape(-1,1)
scaler = StandardScaler()
new_up = scaler.fit_transform(uppercase)
print('time elapsed: ', time.time()-s)

print('-------choose specific stop words-------')
s=time.time()
x_lower = [sublist.lower() for sublist in x_total]
# punctuation = string.punctuation.replace
# regex = re.compile('[' +re.escape(punctuation) +']')
def tokenize(text):
#     text = regex.sub(" ", text) # remove punctuation
    text = text.replace('.',' ')
#     text = text.replace(',',' ')
#     text = text.replace('&',' and ')
#     text = text.replace('/',' or ')
    text = text.split()
    return(text)
x_lower = [tokenize(i) for i in x_lower]
x_unlist = []
for i in x_lower:
    x_unlist += i
vocab_dic = Counter(x_unlist)
stopwords_num = 250
# print([i[0] for i in vocab_dic.most_common(stopwords_num)],'\n\n')
print('total vocab: ',len(vocab_dic.most_common()))
maxfeature = len([i[0] for i in vocab_dic.most_common() if i[1]>0])
print('vocab size frequency > 1: ', maxfeature)

stop = [i[0] for i in vocab_dic.most_common(stopwords_num)]
from itertools import compress
x_5level = [list(compress(x_lower, list(np.array(y_total)==i))) for i in np.unique(y_total)]

x_5level_unlist = [[],[],[],[],[],[],[],[],[],[],[]]
for i in range(len(x_5level)):
    for j in x_5level[i]:
        x_5level_unlist[i] += j
multilevel_vocab = []
for i in range(11):
    multilevel_vocab.append(Counter(x_5level_unlist[i]))


from collections import defaultdict
multilevel_stop = defaultdict(list)
for i in range(11):
    tt = len(x_5level_unlist[i])
    for j in stop:
        multilevel_stop[j].append(multilevel_vocab[i][j]/tt)

stop_var = [(key,np.std(value)*1000) for key,value in multilevel_stop.items() ]
stop = [i[0] for i in stop_var if i[1]<0.1]

print('time elapsed: ', time.time()-s)



def tokenize(text):
    try:
        punctuation = string.punctuation.replace('#','')
#         .replace('!','').replace('?','').replace("'",'').replace("/",'').replace("@",'').replace('"','')
        regex = re.compile('[' +re.escape(punctuation) +']')
        text = regex.sub(" ", text) # remove punctuation
        text = text.replace('#1','')
        text = text.replace('#2','')
        text = text.replace('#3','')
        text = text.replace('#4','')
        text = text.replace('#5','')
        text = text.replace('#6','')
        text = text.replace('#7','')
        text = text.replace('#8','')
        text = text.replace('#9','')
#         text = text.replace('/w',' with ')
#         text = text.replace('/',' ')
#         text = text.replace('&',' and ')
#         text = text.replace("/",' or ')
#         text = text.replace('""',' sarcasm ')
#         text = text.replace("'d",' ')
#         text = text.replace("'s",' ')
#         text = text.replace("'re",' ')
#         text = text.replace("'ll",' ')
#         text = text.replace("'ve",' ')    

#         text = text.replace('.',' ')

        ps = PorterStemmer()
        tokens = []
        tokens_ = [s.split() for s in sent_tokenize(text)]
        for token_by_sent in tokens_:
            tokens += token_by_sent   
        filtered_tokens = [ps.stem(w.lower()) for w in tokens]
        return filtered_tokens
    except TypeError as e: print(text,e)

print('-------transform to features-------')
s=time.time()
onehot_list = ['age_cat', 'sex', 'stay_cat', 'lang', 'er','age_cat_sex', 'age_cat_stay_cat',
       'age_cat_lang', 'age_cat_er', 'age_cat_category', 'sex_stay_cat',
       'sex_lang', 'sex_er', 'sex_category', 'stay_cat_lang', 'stay_cat_er',
       'stay_cat_category', 'lang_er', 'lang_category', 'er_category']
onehot_pipeline = make_pipeline(FeatureExtractor(onehot_list),
                                CategoricalEncoding('OneHot'),
                                )
descrip_pipeline = make_pipeline(FeatureExtractor('comment'),
                                text(method='tfidf', ngram = 3, max_f = maxfeature, 
                                     binary = False, stopwords=stop,tokenizer=tokenize,analyzer ='word'))

feature_union = make_union(
    onehot_pipeline,
    descrip_pipeline
)
X = feature_union.fit_transform(train)
print('time elapsed: ', time.time()-s)

print('-------add length and upper prob to features-------')
s = time.time()
length = np.array([(X[i,]!=0).sum() for i in range(X.shape[0])]).reshape(-1,1)
scaler = StandardScaler()
new_l = scaler.fit_transform(length)
X = hstack([X,new_l],format='csr')
X = hstack([X,new_up],format='csr')

print('X shape: ',X.shape)
print('time elapsed: ', time.time()-s)

x_test=X

print('--------------Word ridge data complete--------------')

/home/kevindong1994/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


-------generate interaction feature between cate-------
time elapsed:  7.466782093048096
-------calculate uppercase prob to features-------
time elapsed:  0.23914670944213867
-------choose specific stop words-------
total vocab:  17572
vocab size frequency > 1:  17572
time elapsed:  0.3989870548248291
-------transform to features-------
time elapsed:  11.973384380340576
-------add length and upper prob to features-------
X shape:  (26954, 17997)
time elapsed:  5.664021730422974
--------------Word ridge data complete--------------


/home/kevindong1994/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


# Model

In [ ]:
print('-------Ridge-------')
s = time.time()
# rkf = RepeatedKFold(n_splits=5, n_repeats=1)
# parameters = {'alpha':[1.3,1.5,10]}
# mse_score = make_scorer(mean_squared_error)
# ridge_model = RidgeClassifier()
# ridge_cv = GridSearchCV(ridge_model, parameters,cv=rkf, pre_dispatch=2, return_train_score = True,scoring=mse_score)
# ridge_cv.fit(x_train, y_train)
# print(ridge_cv.cv_results_)
# ridge_train_res = ridge_cv.predict(x_train)
# ridge_test_res = ridge_cv.predict(x_test)
ridge_model = Ridge(alpha = 1.5)
ridge_model = ridge_model.fit(x_train, y_train)
ridge_train_res = ridge_model.predict(x_train)
ridge_test_res = ridge_model.predict(x_test)
joblib.dump(ridge_test_res, "ridge_test_res.pkl")


ridge_train_res1 = [10 if i >10 else round(i) for i in ridge_train_res]
ridge_train_res1 = np.array([0 if i<0 else i for i in ridge_train_res1])
ridge_test_res1 = [10 if i >10 else round(i) for i in ridge_test_res]
ridge_test_res1 = np.array([0 if i<0 else i for i in ridge_test_res1])

print("train accuracy:", mean_squared_error(ridge_train_res1, y_train))
print("test accuracy:", mean_squared_error(ridge_test_res1, y_test))
print('time elapsed: ', time.time()-s)
r_w = ridge_test_res

In [4]:
import mord
classifier = mord.OrdinalRidge(max_iter=100000,solver="sag",tol=0.001)
classifier.fit(x_train,y_train)
prediction = classifier.predict(x_test)


ValueError: dimension mismatch

In [7]:
x_train.shape,x_test.shape

((151824, 45760), (26954, 17997))